# Lab Supply Calculator

Welcome! Use this tool to determine the quantity of supplies you will need for a given sample size, compare that to the current inventory list, generate a report of supply needs and reorder status, and then update the inventory list with the supply needs. This tool requires access to the inventory list (a shared Google sheet), a lab protocol (saved in GitHub and formatted in yaml), and a user-defined sample size.
<br>
<br>
Please make sure you have access to the Inventory Google sheet prior to starting. The inventory list is located at the following link: https://docs.google.com/spreadsheets/d/1I1q2VBpyYtZItV5uP4WE2zGC5sm3qWFAK2D8yQtntFg/edit?usp=sharing.
<br>
<br>
The protocols are currently being stored in the following repository: https://github.com/aono87/inventory_test/tree/main/test_protocol_check/protocols
<br>
<br>
If you are unable to access the sheet, please contact Vicki Pease (vicki.pease@noaa.gov).

If you have any problem running the program, please contact Aubrie Onoufriou (aubrie.onoufriou@noaa.gov)
<br>
<br>

# Instructions
1. Press "Run all" to get started.

    Note: It may take several seconds to install and update the required packages.

    Note: You will get a pop up to authenticate yourself the first time this is run.

2. Enter your name (first initial and last name, e.g., AOnoufriou).

3. Enter the number of samples and number of plates.

4. Choose the protocol from the dropdown menu.

5. Click "Generate Report".

6. When you are happy with the report, you can save the report to your computer as a CSV by clicking "Download CSV".

7. Press "Update Inventory Sheet" to add the required amounts, protocol and user information to the Inventory Google sheet document.

**Note about Google credentials:**
This notebook is run through the Google environment and therefore will use your same NOAA Google credentials. As long as you have permission to access the "Inventory" Google sheet, this program will run.

**Troubleshooting:**
You can clear all output and start clean by going to "Edit", then "Clear all outputs".
Sometimes the program gets clunky when you update the report several times. This can cause two CSV reports to download. If this starts to happen, clear everything (include the packages that have been installed) by going to "Runtime", then  "Disconnect and delete runtime". You can then try again by pressing  "Run all".

In [1]:
#@title Install packages and imports
##Install packages and imports
# Installing packages
import subprocess
import sys
import re

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import yaml
except ImportError:
    install_package("pyyaml")
import yaml

try:
    import gspread
except ImportError:
    install_package("gspread")

# Ensure these are installed for Google Colab authentication with gspread
try:
    import google.auth
except ImportError:
    install_package("google-auth")
try:
    import google.auth.transport.requests
except ImportError:
    install_package("google-auth-httplib2")

# NEW: Install the pytz library for timezone handling
try:
    import pytz
except ImportError:
    install_package("pytz")
import pytz


# Import for Google Colab authentication
try:
    from google.colab import auth
except ImportError:
    print("Warning: google.colab.auth not found. Running outside Colab might require manual authentication setup for gspread.")


# Import all required libraries
import requests
import csv
import pandas as pd
import os
import math # Added for ceiling function
from io import StringIO
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
from datetime import datetime # Import the datetime object


print("✅ All packages installed and imported successfully!")

# --- Google Sheet Authentication Sanity Check ---
global gc # Make gc global so it can be used in load_inventory
try:
    print("\n🔑 Authenticating Google Colab for Sheets access (this may open a pop-up)...")
    auth.authenticate_user()
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)
    print("✅ Google Sheets authentication successful!")
except Exception as e:
    print(f"❌ Initial Google Sheets authentication failed: {e}")
    print("Please ensure you complete the authentication pop-up and have proper sheet permissions.")
    gc = None # Set gc to None if authentication fails
# ----------------------------------------------------------

# Define GitHub repo configuration
GITHUB_USER = "aono87"
REPO_NAME = "inventory_test"
BRANCH = "main"
PROTOCOL_DIR = "test_protocol_check/protocols"
GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/1I1q2VBpyYtZItV5uP4WE2zGC5sm3qWFAK2D8yQtntFg/edit?usp=sharing"

print("✅ Configuration set")

#Helper Functions
def github_raw_url(path):
    return f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/{path}"

def fetch_file(path):
    url = github_raw_url(path)
    try:
        r = requests.get(url)
        r.raise_for_status()
        return r.text
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {path}: {e}")
        raise

def get_protocol_list(user, repo, path, branch="main"):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}"
    try:
        r = requests.get(url)
        r.raise_for_status()
        files = r.json()
        return [f['path'] for f in files if f['name'].endswith('.yaml')]
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching protocol list: {e}")
        return []

def get_project_list(user, repo, path, branch="main"):
    """
    Fetches a list of directories from a GitHub repository path using a two-step
    process to work around API inconsistencies for some repositories.
    """
    # Step 1: Get the contents of the repository's root to find the folder's unique ID (SHA)
    root_contents_url = f"https://api.github.com/repos/{user}/{repo}/contents/?ref={branch}"
    try:
        r_root = requests.get(root_contents_url)
        r_root.raise_for_status()
        root_contents = r_root.json()

        # Find the SHA for the target path (e.g., 'Active')
        folder_sha = None
        for item in root_contents:
            if item['name'] == path and item['type'] == 'dir':
                folder_sha = item['sha']
                break

        if not folder_sha:
            print(f"❌ Could not find folder '{path}' in the repository root.")
            return []

        # Step 2: Use the Git Trees API with the folder's SHA to get its contents
        tree_url = f"https://api.github.com/repos/{user}/{repo}/git/trees/{folder_sha}"
        r_tree = requests.get(tree_url)
        r_tree.raise_for_status()
        tree_contents = r_tree.json()

        # Filter for items that are directories ('tree') and get their names ('path')
        project_folders = [item['path'] for item in tree_contents.get('tree', []) if item['type'] == 'tree']
        return project_folders

    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching project list: {e}")
        return []

def load_inventory():
    if gc is None:
        print("❌ Google Sheets client not initialized. Cannot load inventory.")
        return pd.DataFrame() # Return empty DataFrame on failure
    try:
        spreadsheet = gc.open_by_url(GOOGLE_SHEET_URL)
        worksheet = spreadsheet.get_worksheet(0)
        records = worksheet.get_all_records()
        df = pd.DataFrame(records)

        # Clean up column names to prevent errors
        df.columns = df.columns.str.strip()

        # Ensure numeric columns are treated as numbers, handling empty strings
        # FIX: Include the new 'Stock Unit Quantity-Live' column
        numeric_cols = ['Amount per stock unit (In protocol unit)', 'Stock Unit Quantity', 'Stock Unit Quantity-Live']
        for col in numeric_cols:
            # Check if the column exists before trying to convert
            if col in df.columns:
                 df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            else:
                 print(f"⚠️ Warning: Column '{col}' not found in the inventory sheet.")
                 # Add the column with default 0 if it's missing to prevent errors later
                 df[col] = 0


        print("✅ Inventory loaded successfully from Google Sheet into DataFrame.")
        return df
    except Exception as e:
        print(f"❌ Error loading inventory from Google Sheet: {e}")
        return pd.DataFrame()

def load_protocol(text):
    try:
        protocol_data = yaml.safe_load(text)
        return protocol_data
    except Exception as e:
        print(f"❌ Error loading protocol: {e}")
        return {}

def calculate_needs(protocol_data, sample_count, plate_count):
    """Calculates total needs and returns a DataFrame, preserving duplicate items."""
    needs_list = []

    # Handle per-sample supplies
    # Assume supplies_per_sample is a list of dictionaries if duplicates are intended
    sample_supplies = protocol_data.get('supplies_per_sample')
    if isinstance(sample_supplies, list):
        for item_details in sample_supplies:
            # Ensure item_details is a dictionary and has 'item' and 'quantity' keys
            if isinstance(item_details, dict) and 'item' in item_details and 'quantity' in item_details:
                item = item_details['item']
                qty = item_details['quantity']
                unit = item_details.get('unit', 'N/A')
                needs_list.append({
                    'Item': item,
                    'Required per unit': f"{qty} {unit}/sample",
                    'Total needed (protocol unit)': qty * sample_count,
                    'protocol_unit': unit
                })
            elif isinstance(item_details, dict) and len(item_details) == 1:
                 # Handle case where format is just item_name: quantity/unit
                 item, details = list(item_details.items())[0]
                 if isinstance(details, dict):
                     qty = details.get('quantity', details)
                     unit = details.get('unit', 'N/A')
                 else:
                     qty = details
                     unit = 'N/A'
                 needs_list.append({
                    'Item': item,
                    'Required per unit': f"{qty} {unit}/sample",
                    'Total needed (protocol unit)': qty * sample_count,
                    'protocol_unit': unit
                })

    elif isinstance(sample_supplies, dict): # Handle existing dictionary format
        for item, details in sample_supplies.items():
            qty = details.get('quantity', details) if isinstance(details, dict) else details
            unit = details.get('unit', 'N/A') if isinstance(details, dict) else 'N/A'
            # Append details for each item from the protocol
            needs_list.append({
                'Item': item,
                'Required per unit': f"{qty} {unit}/sample",
                'Total needed (protocol unit)': qty * sample_count,
                'protocol_unit': unit
            })


    # Handle per-plate supplies
    # Assume supplies_per_plate is a list of dictionaries if duplicates are intended
    plate_supplies = protocol_data.get('supplies_per_plate')
    if isinstance(plate_supplies, list):
        for item_details in plate_supplies:
             # Ensure item_details is a dictionary and has 'item' and 'quantity' keys
            if isinstance(item_details, dict) and 'item' in item_details and 'quantity' in item_details:
                item = item_details['item']
                qty = item_details['quantity']
                unit = item_details.get('unit', 'N/A')
                needs_list.append({
                    'Item': item,
                    'Required per unit': f"{qty} {unit}/plate",
                    'Total needed (protocol unit)': qty * plate_count,
                    'protocol_unit': unit
                })
            elif isinstance(item_details, dict) and len(item_details) == 1:
                 # Handle case where format is just item_name: quantity/unit
                 item, details = list(item_details.items())[0]
                 if isinstance(details, dict):
                     qty = details.get('quantity', details)
                     unit = details.get('unit', 'N/A')
                 else:
                     qty = details
                     unit = 'N/A'
                 needs_list.append({
                    'Item': item,
                    'Required per unit': f"{qty} {unit}/plate",
                    'Total needed (protocol unit)': qty * plate_count,
                    'protocol_unit': unit
                })

    elif isinstance(plate_supplies, dict): # Handle existing dictionary format
        for item, details in plate_supplies.items():
            qty = details.get('quantity', details) if isinstance(details, dict) else details
            unit = details.get('unit', 'N/A') if isinstance(details, dict) else 'N/A'
            # Append details for each item from the protocol
            needs_list.append({
                'Item': item,
                'Required per unit': f"{qty} {unit}/plate",
                'Total needed (protocol unit)': qty * plate_count,
                'protocol_unit': unit
            })


    if not needs_list:
        print("❌ Protocol missing 'supplies_per_sample' and/or 'supplies_per_plate' section or they are empty/malformed.")
        return pd.DataFrame() # Return empty DataFrame if no needs are calculated

    # Return DataFrame from the list of needs, which inherently handles duplicates with different rows
    return pd.DataFrame(needs_list)


print("✅ Helper functions defined")

✅ All packages installed and imported successfully!

🔑 Authenticating Google Colab for Sheets access (this may open a pop-up)...
✅ Google Sheets authentication successful!
✅ Configuration set
✅ Helper functions defined


In [2]:
# @title Generate Supply Report & Update Sheet
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
import os
import csv
import gspread
from datetime import datetime
import pytz
import math
import numpy as np

# This is the "initialize-once" pattern.
# The code inside this 'if' block will only run ONCE per session,
# preventing event handlers from stacking up on re-runs.
if 'calculator_app' not in globals():
    print("🚀 Initializing Lab Supply Calculator for the first time...")
    print("=" * 50)

    # --- Step 1: Create all widgets just once ---
    name_input = widgets.Text(value='AOnoufriou', placeholder='e.g., AOnoufriou', description='Your Name:', style={'description_width': 'initial'})
    samples_input = widgets.IntText(value=96, description='Number of Samples:', style={'description_width': 'initial'})
    plates_input = widgets.IntText(value=1, description='Number of Plates:', style={'description_width': 'initial'})
    protocol_dropdown = widgets.Dropdown(description='Protocol:', layout=widgets.Layout(width='95%'), style={'description_width': 'initial'})

    # NEW: Create the Project dropdown widget
    project_dropdown = widgets.Dropdown(description='Project:', layout=widgets.Layout(width='95%'), style={'description_width': 'initial'})


    generate_button = widgets.Button(description='Generate Report', button_style='primary', tooltip='Click to generate the supply report', layout=widgets.Layout(width='200px', margin='0 10px 0 0'))
    update_sheet_button = widgets.Button(description='Update Inventory Sheet', button_style='warning', tooltip='Writes the hold information to new columns in the Google Sheet', layout=widgets.Layout(width='200px', margin='0 10px 0 0', display='none'))
    download_button = widgets.Button(description='Download CSV', button_style='success', tooltip='Click to download the report as a CSV file', layout=widgets.Layout(width='200px', display='none'))
    output_area = widgets.Output()

    # This dictionary will live inside the app structure
    report_data_for_csv = {}

    # --- Step 2: Define all handler functions just once ---
    def generate_report(b):
        """Calculates and displays the supply report using a merged DataFrame."""
        with output_area:
            output_area.clear_output(wait=True)
            update_sheet_button.layout.display = 'none'
            download_button.layout.display = 'none'
            generate_button.disabled = True
            generate_button.description = "Generating..."
            try:
                print("🔄 Generating report...")
                # 1. Load data into DataFrames
                inventory_df = load_inventory()

                if inventory_df.empty:
                    raise ValueError("Inventory data could not be loaded. Please check logs for a Google Sheet error.")

                protocol_text = fetch_file(protocol_dropdown.value)
                protocol = load_protocol(protocol_text)
                needs_df = calculate_needs(protocol, samples_input.value, plates_input.value)

                if needs_df.empty:
                     print("❌ Protocol data could not be loaded or protocol is empty.")
                     return # Exit function if needs_df is empty

                print(f"✅ Calculated needs for {len(needs_df)} items")

                # 2. Merge the two DataFrames - use left merge to keep all items from protocol
                # This merge will match each row in needs_df with the corresponding row(s) in inventory_df
                # If an item appears multiple times in needs_df, it will attempt to merge each instance.
                # If an item appears multiple times in inventory_df, and once in needs_df, it will create multiple rows.
                # If an item appears once in inventory_df, and multiple times in needs_df, it will repeat the inventory data.
                merged_df = pd.merge(needs_df, inventory_df, on='Item', how='left')

                # 3. Perform calculations on the merged DataFrame - use the correct capitalized column name
                # Fill missing 'Amount per stock unit (In protocol unit)' and 'Stock Unit Quantity' with 0 for calculation
                # This prevents errors for items not found in inventory
                merged_df['Amount per stock unit (In protocol unit)'] = pd.to_numeric(merged_df['Amount per stock unit (In protocol unit)'], errors='coerce').fillna(0)
                # FIX: Use the 'Stock Unit Quantity-Live' column for calculations
                merged_df['Stock Unit Quantity-Live'] = pd.to_numeric(merged_df['Stock Unit Quantity-Live'], errors='coerce').fillna(0)


                # Calculate Total needed (stock unit)
                # Handle division by zero for items with 0 stock unit amount
                merged_df['Total needed (stock unit)'] = np.where(
                    merged_df['Amount per stock unit (In protocol unit)'] > 0,
                    merged_df['Total needed (protocol unit)'] / merged_df['Amount per stock unit (In protocol unit)'],
                    np.nan # Assign NaN if Amount per stock unit is 0
                )

                # Calculate Sufficient Stock
                # FIX: Use 'Stock Unit Quantity-Live' for Sufficient Stock calculation
                merged_df['Sufficient Stock'] = np.where(
                    merged_df['Stock Unit Quantity-Live'] >= merged_df['Total needed (stock unit)'],
                    'YES',
                    'NO'
                )
                # Handle cases where Total needed (stock unit) is NaN (e.g., item not in inventory or Amount per stock unit is 0)
                merged_df['Sufficient Stock'] = merged_df['Sufficient Stock'].fillna('MISSING/N/A')


                # 4. Format columns for display
                # Ensure these columns exist before formatting
                if 'Total needed (protocol unit)' in merged_df.columns:
                    merged_df['Total needed (protocol unit)'] = merged_df.apply(
                        lambda row: f"{round(row['Total needed (protocol unit)'], 3)} {row.get('protocol_unit', 'Unit')}" if pd.notna(row['Total needed (protocol unit)']) else "N/A", axis=1
                    )
                else:
                    merged_df['Total needed (protocol unit)'] = 'N/A - Calculation Error'

                if 'Total needed (stock unit)' in merged_df.columns:
                    merged_df['Total needed (stock unit)'] = merged_df.apply(
                        lambda row: f"{round(row['Total needed (stock unit)'], 3)} {row.get('Stock unit', 'Unit')}(s)" if pd.notna(row['Total needed (stock unit)']) else "N/A", axis=1
                    )
                else:
                    merged_df['Total needed (stock unit)'] = 'N/A - Calculation Error'

                # FIX: Use 'Stock Unit Quantity-Live' for Stock on hand display
                if 'Stock Unit Quantity-Live' in merged_df.columns:
                     merged_df['Stock on hand'] = merged_df.apply(
                        lambda row: f"{round(row['Stock Unit Quantity-Live'], 3)} {row.get('Stock unit', 'Unit')}(s)" if pd.notna(row['Stock Unit Quantity-Live']) else "N/A", axis=1
                    )
                else:
                    merged_df['Stock on hand'] = 'N/A - Inventory Data Missing'


                # Rename columns for the final report
                merged_df.rename(columns={'Expiration Date': 'Expiration date'}, inplace=True)

                # 5. Select and order columns for the final report
                report_columns = [
                    "Item", "Required per unit", "Total needed (protocol unit)",
                    "Total needed (stock unit)", "Stock on hand",
                    "Sufficient Stock", "Expiration date"
                ]

                # Ensure all report_columns exist in the merged_df before selecting
                existing_report_columns = [col for col in report_columns if col in merged_df.columns]
                final_report_df = merged_df[existing_report_columns].fillna('N/A')

                report_data_for_csv['df'] = final_report_df
                report_data_for_csv['user_name'] = name_input.value
                report_data_for_csv['protocol_name'] = os.path.basename(protocol_dropdown.value)
                report_data_for_csv['sample_size'] = samples_input.value
                report_data_for_csv['plate_count'] = plates_input.value
                # NEW: Store the selected project name
                report_data_for_csv['project_name'] = project_dropdown.value


                print(f"\n📋 Supply Report for {samples_input.value} samples and {plates_input.value} plates by {name_input.value}")
                # NEW: Display the selected project name in the report header
                print(f"Project: {report_data_for_csv['project_name']}")
                print("=" * 80)
                display(final_report_df.style.hide(axis="index"))

                update_sheet_button.disabled = False
                update_sheet_button.description = 'Update Inventory Sheet'
                update_sheet_button.layout.display = ''
                download_button.disabled = False
                download_button.description = 'Download CSV'
                download_button.layout.display = ''
            except Exception as e:
                print(f"❌ Error during report generation: {e}")
            finally:
                generate_button.disabled = False
                generate_button.description = "Generate Report" # Reset button to its initial state

    def update_inventory_sheet(b):
        """Writes the hold information to the Google Sheet with correct timezone."""
        if b.disabled: return
        b.disabled = True
        with output_area:
            print("\n🔄 Updating Google Sheet with hold information...")
            b.description = "Updating..."
            try:
                report_df = report_data_for_csv.get('df')
                user_name = report_data_for_csv.get('user_name', 'N/A')
                protocol_name = report_data_for_csv.get('protocol_name', 'N/A')
                # Get the project name for the update
                project_name = report_data_for_csv.get('project_name', 'N/A')

                spreadsheet = gc.open_by_url(GOOGLE_SHEET_URL)
                worksheet = spreadsheet.get_worksheet(0)
                headers = worksheet.row_values(1)
                inventory_data = worksheet.get_all_records()
                item_to_row_map = {str(record['Item']).strip(): i + 2 for i, record in enumerate(inventory_data)}
                next_col = len(headers) + 1

                pacific_tz = pytz.timezone('America/Los_Angeles')
                now_pacific = datetime.now(pacific_tz)
                date_str = now_pacific.strftime('%Y-%m-%d %I:%M %p %Z')

                unique_suffix = f"{user_name} on {date_str}"
                new_headers = [f"Hold For ({unique_suffix})", f"Hold Amount ({unique_suffix})", f"Hold Date ({unique_suffix})", f"Hold Project ({unique_suffix})"]
                header_update_range = f'{gspread.utils.rowcol_to_a1(1, next_col)}:{gspread.utils.rowcol_to_a1(1, next_col + 3)}'
                worksheet.update(range_name=header_update_range, values=[new_headers])
                print(f"✅ Added new headers starting in column {gspread.utils.rowcol_to_a1(1, next_col).strip('1')}.")
                report_date_str = now_pacific.strftime('%Y-%m-%d')
                updates = []
                for _, row in report_df.iterrows():
                    # Extract the numeric part of 'Total needed (stock unit)' for the hold amount
                    item_name = str(row['Item']).strip()
                    needed_str = str(row['Total needed (stock unit)'])
                    if item_name in item_to_row_map and pd.notna(needed_str) and needed_str != 'N/A':
                        needed_amount = needed_str.split(' ')[0]
                        # Find all rows in the original inventory data that match the item name
                        matching_inventory_rows = [i + 2 for i, record in enumerate(inventory_data) if str(record['Item']).strip() == item_name]

                        if matching_inventory_rows:
                            # For simplicity, update the first matching row found in the inventory sheet.
                            # If the user needs to specify which row to update for duplicates in the sheet,
                            # the inventory sheet structure or the update logic would need to be more complex.
                            row_idx = matching_inventory_rows[0]
                            updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, next_col)}', 'values': [[user_name]]})
                            updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, next_col + 1)}', 'values': [[str(needed_amount)]]})
                            updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, next_col + 2)}', 'values': [[report_date_str]]})
                            # Include the project name in the update
                            updates.append({'range': f'{gspread.utils.rowcol_to_a1(row_idx, next_col + 3)}', 'values': [[project_name]]})
                        else:
                             print(f"🤷 No matching item '{item_name}' found in the inventory sheet to update.")

                if updates:
                    # Split batch update into smaller chunks if necessary (gspread limit is 5000 cells per request)
                    # A simple approach is to update one row at a time if batching causes issues,
                    # but batching is generally more efficient. Let's stick with batching for now.
                    worksheet.batch_update(updates)
                    print(f"✅ Successfully updated hold information for {len(updates)//4} items.")
                    b.description = 'Updated ✓'
                else:
                    print("🤷 No items in the report matched the inventory sheet. Nothing to update.")
                    b.description = 'Nothing to Update'
            except Exception as e:
                print(f"❌ Sheet update failed: {e}")
                b.disabled = False
                b.description = 'Update Failed'


    def download_csv(b):
        """Creates and downloads the report as a CSV file, now with new columns."""
        if b.disabled: return
        b.disabled = True
        with output_area:
            print("\n💾 Preparing CSV for download...")
            b.description = "Downloading..."
            try:
                df = report_data_for_csv.get('df')
                user_name = report_data_for_csv.get('user_name', 'N/A')
                protocol_name = report_data_for_csv.get('protocol_name', 'N/A')
                sample_size = report_data_for_csv.get('sample_size', 'N/A')
                plate_count = report_data_for_csv.get('plate_count', 'N/A')
                 # NEW: Get the project name for the CSV metadata
                project_name = report_data_for_csv.get('project_name', 'N/A')

                protocol_base_name = os.path.splitext(protocol_name)[0].replace(' ', '_')

                pacific_tz = pytz.timezone('America/Los_Angeles')
                now_pacific = datetime.now(pacific_tz)
                date_str = now_pacific.strftime('%Y-%m-%d')
                time_str = now_pacific.strftime('%Y-%m-%d %I:%M:%S %p %Z')

                filename = f"Report_{protocol_base_name}_{sample_size}-samples_{plate_count}-plates_{user_name}_{date_str}.csv"

                # Add new columns to metadata
                # Determine the number of report columns to pad the metadata lists
                num_report_cols = len(df.columns) if not df.empty else 7 # Default to 7 if df is empty
                empty_padding = [''] * (num_report_cols - 1) # Number of empty strings needed to match report columns

                metadata_rows = {
                    'Item': [
                        f"User: {user_name}",
                        f"Protocol: {protocol_name}",
                        f"Sample Size: {sample_size}",
                        f"Plate Count: {plate_count}",
                        f"Project: {project_name}", # Add the project name metadata row
                        f"Generated: {time_str}"
                        ]
                }
                # Add empty padding for other columns in metadata rows
                for col in df.columns:
                    if col != 'Item':
                        metadata_rows[col] = [''] * len(metadata_rows['Item'])


                metadata = pd.DataFrame(metadata_rows)
                final_output = pd.concat([metadata, df], ignore_index=True)
                final_output.to_csv(filename, index=False)
                print(f"📁 Starting download for: {filename}")
                files.download(filename)
                print("🎉 Download complete!")
                b.description = 'Downloaded ✓'
            except Exception as e:
                print(f"❌ Download failed: {e}")
                b.disabled = False
                b.description = "Download Failed"

    # --- Step 3: Connect handlers and populate dropdown just once ---
    generate_button.on_click(generate_report)
    update_sheet_button.on_click(update_inventory_sheet)
    download_button.on_click(download_csv)

    with output_area:
        try:
            print("🔍 Fetching available protocols from GitHub...")
            protocols = get_protocol_list(GITHUB_USER, REPO_NAME, PROTOCOL_DIR, BRANCH)
            if not protocols: raise ValueError("No protocols found on GitHub.")
            protocol_dropdown.options = protocols
            print(f"✅ Found {len(protocols)} protocols.")
        except Exception as e:
            print(f"❌ {e} Using a default.")
            protocol_dropdown.options = ["test_protocol_check/protocols/dna_extraction_mn.yaml"]

        # NEW: Fetch and populate Project dropdown
        try:
            print("🔍 Fetching available projects from GitHub...")
            project_user = "SWFSC-MMTD-Genetics"
            project_repo = "MMTD-Genetics-Projects"
            project_path = "Active"
            projects = get_project_list(project_user, project_repo, project_path, BRANCH)
            if not projects: raise ValueError("No projects found on GitHub.")
            project_dropdown.options = projects
            print(f"✅ Found {len(projects)} projects.")
        except Exception as e:
            print(f"❌ {e} Using a default.")
            project_dropdown.options = ["Default Project"]


    # --- Step 4: Assemble the UI and store it in a global variable ---
    buttons_box = widgets.HBox([generate_button, update_sheet_button, download_button])
    globals()['calculator_app'] = widgets.VBox([
        widgets.HTML("<b>Please enter your details below and click 'Generate Report'.</b>"),
        name_input,
        samples_input,
        plates_input,
        protocol_dropdown,
        # NEW: Include the Project dropdown in the UI layout
        project_dropdown,
        buttons_box,
        widgets.HTML("<hr>"),
        output_area
    ])

# --- Display the App ---
# This line runs every time the cell is executed, but it just re-displays the
# single, correctly configured app without adding new, duplicate event handlers.
display(globals()['calculator_app'])

🚀 Initializing Lab Supply Calculator for the first time...


In [3]:
# Access the last selected protocol file path from the dropdown widget
selected_protocol_path = protocol_dropdown.value

if selected_protocol_path:
    print(f"Fetching and printing the content of: {selected_protocol_path}")
    try:
        # Use the fetch_file helper function defined in the first cell
        protocol_content = fetch_file(selected_protocol_path)
        if protocol_content:
            print("```yaml")
            print(protocol_content)
            print("```")
        else:
            print("❌ Failed to fetch protocol content.")
    except Exception as e:
        print(f"❌ An error occurred while fetching the protocol: {e}")
else:
    print("🤷 No protocol file was selected in the dropdown.")

Fetching and printing the content of: test_protocol_check/protocols/ddpcr_mn.yaml
```yaml
# Protocol Name: ddPCR ABO
# Description: A protocol for ddPCR where some supplies are calculated per sample (reaction)
# and others are calculated per plate.

protocol_name: "ddPCR ABO"

# This key indicates that the protocol uses the new calculation logic.
# The presence of 'supplies' (instead of 'supplies_per_sample') triggers the new calculator mode.
supplies:
  'ddPCR 96 well Semi skirted plates':
    quantity: 1
    unit: 'item'
    per: 'plate'

  'BioRad ddPCR Cartridges':
    quantity: 1
    unit: 'item'
    per: 'plate'

  'BioRad ddPCR Gaskets':
    quantity: 1
    unit: 'item'
    per: 'plate'

  'BioRad pierceable foil heat seal':
    quantity: 1
    unit: 'item'
    per: 'plate'

  'BioRad ddPCR Oil for Probes':
    quantity: 70
    unit: 'ul'
    per: 'sample'

  'ddPCR™ Supermix for Residual DNA Quantification (MMGP)':
    quantity: 10
    unit: 'ul'
    per: 'sample'

  'ddPCR sup